### **Paso 2.5 - Ingesta del archivo "results.json"**

#### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [ ]:
results_df = spark.read \
.schema(results_schema) \
.json("/mnt/formula1dl/raw/results.json")

In [ ]:
results_df.show(truncate=False)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|time       |milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|1       |18    |1       |1            |22    |1   |1       |1           |1            |10.0  |58  |1:34:50.616|5690616     |39        |2   |1:27.452      |218.3          |1       |
|2       |18    |2       |2            |3     |5   |2       |2           |2            |8.0   |58  |+5.478     |5696094     |41        |3   |1:27.739      |217.586        |1       |
|3       |18    |3       |3            |7     |7   |3       |3           |3            |6.

#### Paso 2 - Renombrar columnas y añadir columnas nuevas

In [ ]:
from pyspark.sql.functions import lit, current_timestamp

In [ ]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("ingestion_date", current_timestamp())
                                    #.withColumn("data_source", lit(v_data_source)) \
                                    #.withColumn("file_date", lit(v_file_date))

In [ ]:
results_with_columns_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time       |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|statusId|ingestion_date         |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+-----------------------+
|1        |18     |1        |1             |22    |1   |1       |1            |1             |10.0  |58  |1:34:50.616|5690616     |39         |2   |1:27.452        |218.3            |1       |2023-06-10 15:42:33.037|
|2        |18     |2        |2             |3     |5   |2       |2            |2             |8.0   |58  |+5.478     |5696094     |4

#### Paso 3 - Eliminar la columna no deseada

In [ ]:
from pyspark.sql.functions import col

In [ ]:
results_final_df = results_with_columns_df.drop(col("statusId"))

In [ ]:
results_final_df.show(truncate=False)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+-----------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|time       |milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|ingestion_date         |
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+-----------------------+
|1        |18     |1        |1             |22    |1   |1       |1            |1             |10.0  |58  |1:34:50.616|5690616     |39         |2   |1:27.452        |218.3            |2023-06-10 15:43:45.663|
|2        |18     |2        |2             |3     |5   |2       |2            |2             |8.0   |58  |+5.478     |5696094     |41         |3   |1:27.739        |217

#### Paso 4 - Escribir datos en el contenedor **processed** del ADLS como **parquet**

In [ ]:
# Escribimos el archivo con formato PARQUET en el contenedor "processed" y directorio "results"
# Particionamos el archivo según los valores de la columna "race_id"
results_final_df.write.mode('overwrite').partitionBy('race_id').parquet("/mnt/formula1dl/processed/results")

In [ ]:
# Visualizamos los objetos que se encuentran en el directorio "results"
%fs
ls /mnt/formula1dl/processed/results